<a href="https://colab.research.google.com/github/Elma-dev/fuel_calculation/blob/main/fuel_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import json

In [4]:
# LV_data=pd.read_csv("LV3.csv",sep=";") #Read csv data
LV_data = pd.read_excel("LV1.xlsx")

In [5]:
LV_data.head()

,tracking_time,priority,longitude,latitude,altitude,angle,satellites,speed,km,message,...,device_id,vehicle_id,power,battery,volume_tank1,volume_tank2,signal,temperature,mat,code
0,2023-06-19 00:06:44,0,-5.860758,35.75858,97,0,10,0,0,NaN,...,1003651,1003161,13.014,13.014,NaN,NaN,5,NaN,61700-B-40 LV,61700-B-40 LV
1,2023-06-19 00:16:46,0,-5.860758,35.75858,72,0,11,0,0,NaN,...,1003651,1003161,13.014,13.014,NaN,NaN,5,NaN,61700-B-40 LV,61700-B-40 LV
2,2023-06-19 00:26:47,0,-5.860758,35.75858,72,0,12,0,0,NaN,...,1003651,1003161,13.016,13.016,NaN,NaN,5,NaN,61700-B-40 LV,61700-B-40 LV
3,2023-06-19 00:39:06,0,-5.860758,35.75858,90,0,11,0,0,NaN,...,1003651,1003161,13.010,13.010,NaN,NaN,5,NaN,61700-B-40 LV,61700-B-40 LV
4,2023-06-19 00:49:07,0,-5.860758,35.75858,71,0,9,0,0,NaN,...,1003651,1003161,13.013,13.013,NaN,NaN,5,NaN,61700-B-40 LV,61700-B-40 LV


In [6]:
LV_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8718 entries, 0 to 8717
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tracking_time               8718 non-null   datetime64[ns]
 1   priority                    8718 non-null   int64         
 2   longitude                   8718 non-null   float64       
 3   latitude                    8718 non-null   float64       
 4   altitude                    8718 non-null   int64         
 5   angle                       8718 non-null   int64         
 6   satellites                  8718 non-null   int64         
 7   speed                       8718 non-null   int64         
 8   km                          8718 non-null   int64         
 9   message                     0 non-null      float64       
 10  can_speed                   8718 non-null   int64         
 11  accelerator_pedal_position  8718 non-null   int64       

## Check Code Of Fuel Level
---
Function could us to find fuel level code

---

In [7]:
def check_code(LV_data):
  keys=["84","89","48"]
  m,index=-1,-1
  for i,k in enumerate(keys):
    v=[json.loads(row).get(k) for row in LV_data.alv_data if json.loads(row).get(k)!=None]
    v=sum([float(val)!=0 for val in v])
    if m<v:
      m,index=v,i
  return keys[index]

In [8]:
#Test check_code Function
check_code(LV_data)

'84'

## before remove 0 values

In [9]:
# Création d'un dictionnaire pour stocker les valeurs de temps de suivi et les niveaux de carburant
fuel_time = {"tracking_time": [], "fuel_level": []}

# Parcourir les valeurs de la colonne 'alv_data' du DataFrame LV_data
for i, alv in enumerate(LV_data.alv_data.values):
    # Charger la valeur 'alv' sous forme de dictionnaire à partir de JSON
    alv_value = json.loads(alv)

    # Vérifier si la clé "84" existe dans le dictionnaire 'alv_value'
    if alv_value.get("84") is not None:
        # Si la valeur associée à la clé "84" n'est pas zéro, ajouter au dictionnaire fuel_time
        if float(alv_value.get("84")) != 0:
            # Ajouter le temps de suivi à la liste 'tracking_time'
            fuel_time["tracking_time"].append(LV_data.at[i, "tracking_time"])

            # Ajouter le niveau de carburant converti à la liste 'fuel_level'
            fuel_time["fuel_level"].append(float(alv_value.get("84")) * 0.1)

# Création d'un DataFrame à partir du dictionnaire fuel_time
fuel_time_pd = pd.DataFrame(data=fuel_time)

In [10]:
fuel_time_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3974 entries, 0 to 3973
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   tracking_time  3974 non-null   datetime64[ns]
 1   fuel_level     3974 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 62.2 KB


In [11]:
px.line(data_frame=fuel_time_pd,x="tracking_time",y="fuel_level")

## After removing 0 values (Run This)

In [13]:
pd.unique(LV_data.mat) # Show Matricules we have in data

array(['61700-B-40 LV', '78541.B.40 LV'], dtype=object)

In [14]:
def loadData(data, matricule, sizeTotale=0):
    # Code list
    is_percent={"84":False,"89":True,"48":False}
    # Define fuel Level Code
    code=check_code(data)
    #isPercent Value
    percent=is_percent[code]
    # Sélectionner les données du matricule spécifié
    data = data[data.mat == matricule].reset_index()

    # Initialiser un dictionnaire pour stocker les valeurs de temps de suivi et les niveaux de carburant
    fuel_time = {"tracking_time": [], "fuel_level": []}
    j = 0  # Initialisation de la variable j pour stocker la valeur de niveau de carburant

    # Parcourir les données de "alv_data" pour le matricule spécifié
    for i, alv in enumerate(data.alv_data.values):
        alv_value = json.loads(alv)

        # Vérifier si la clé spécifiée (par défaut "84") existe dans le dictionnaire 'alv_value'
        if alv_value.get(code) is not None:
            # Vérifier si la clé spécifiée (par défaut "84") existe dans le dictionnaire 'alv_value'

            if float(alv_value.get(code)) == 0:
                # Si la valeur associée à la clé est zéro

                fuel_time["tracking_time"].append(data.at[i, "tracking_time"])
                # Ajouter le temps de suivi à la liste 'tracking_time'

                if percent:
                    # Si le paramètre 'percent' est True

                    fuel_time["fuel_level"].append(j * sizeTotale / 100)
                    # Ajouter la valeur de carburant (j) multipliée par le pourcentage de 'sizeTotale'
                else:
                    fuel_time["fuel_level"].append(j * 0.1)
                    # Sinon, ajouter la valeur de carburant (j) multipliée par 0.1 (pour obtenir des litres)

            elif float(alv_value.get(code)) != 0:
                # Si la valeur associée à la clé n'est pas zéro

                j = float(alv_value.get(code))
                # Stocker temporairement la valeur de carburant (j)

                fuel_time["tracking_time"].append(data.at[i, "tracking_time"])
                # Ajouter le temps de suivi à la liste 'tracking_time'

                if not percent:
                    # Si le paramètre 'percent' est False

                    fuel_time["fuel_level"].append(float(alv_value.get(code)) * 0.1)
                    # Ajouter la valeur de carburant convertie en litres
                else:
                    fuel_time["fuel_level"].append(float(alv_value.get(code)) * sizeTotale / 100)
                    # Sinon, ajouter la valeur de carburant convertie en pourcentage de 'sizeTotale'

    # Créer un DataFrame à partir du dictionnaire fuel_time
    fuel_time_pd = pd.DataFrame(data=fuel_time)

    # Retourner le DataFrame avec les données de temps de suivi et de niveaux de carburant
    return fuel_time_pd


In [16]:
# Utilisation de la fonction loadData pour extraire les données de carburant spécifiques
fuel_time_pd=loadData(LV_data,matricule="78541.B.40 LV")
# fuel_time_pd=loadData(LV_data,matricule="10535-A-36 LV",code="89",percent=True,sizeTotale=400)
# Affichage des premières lignes du DataFrame fuel_time_pd

fuel_time_pd.head()

,tracking_time,fuel_level
0,2023-06-19 00:04:01,0.0
1,2023-06-19 00:14:02,0.0
2,2023-06-19 00:24:03,0.0
3,2023-06-19 00:34:04,0.0
4,2023-06-19 00:44:05,0.0


In [17]:
px.line(data_frame=fuel_time_pd,x="tracking_time",y="fuel_level")

In [ ]:
# sns.lineplot(data=fuel_time_pd,x="tracking_time",y="fuel_level")

## Fuel Size Versus Angle

In [ ]:
# Initialisation d'un dictionnaire pour stocker les valeurs de temps de suivi, angles et niveaux de carburant
fuel_time = {"tracking_time": [], "fuel_level": [], "angle": []}

# Parcourir les données de "alv_data" dans le DataFrame LV_data
for i, alv in enumerate(LV_data.alv_data.values):
    alv_value = json.loads(alv)

    # Vérifier si la clé "84" existe dans le dictionnaire 'alv_value'
    if alv_value.get("84") is not None:
        # Vérifier si la valeur associée à la clé "84" n'est pas zéro
        if float(alv_value.get("84")) != 0:
            fuel_time["tracking_time"].append(LV_data.at[i, "tracking_time"])
            fuel_time["angle"].append(LV_data.at[i, "angle"])
            fuel_time["fuel_level"].append(float(alv_value.get("84")) * 0.1)

# Création d'un DataFrame à partir du dictionnaire fuel_time
fuel_time_pd = pd.DataFrame(data=fuel_time)

In [ ]:
fuel_time_pd

,tracking_time,fuel_level,angle


In [ ]:
px.line(data_frame=fuel_time_pd,x="tracking_time",y="fuel_level",text="angle")

## Add Altitude

In [ ]:
# Initialisation d'un dictionnaire pour stocker les valeurs de temps de suivi, angles, altitudes et niveaux de carburant
fuel_time = {"tracking_time": [], "fuel_level": [], "angle": [], "altitude": []}

# Parcourir les données de "alv_data" dans le DataFrame LV_data
for i, alv in enumerate(LV_data.alv_data.values):
    alv_value = json.loads(alv)

    # Vérifier si la clé "84" existe dans le dictionnaire 'alv_value'
    if alv_value.get("84") is not None:
        # Vérifier si la valeur associée à la clé "84" n'est pas zéro
        if float(alv_value.get("84")) != 0:
            fuel_time["tracking_time"].append(LV_data.at[i, "tracking_time"])
            fuel_time["angle"].append(LV_data.at[i, "angle"])
            fuel_time["altitude"].append(LV_data.at[i, "altitude"])
            fuel_time["fuel_level"].append(float(alv_value.get("84")) * 0.1)

# Création d'un DataFrame à partir du dictionnaire fuel_time
fuel_time_pd = pd.DataFrame(data=fuel_time)

In [ ]:
fuel_time_pd

,tracking_time,fuel_level,angle,altitude


In [ ]:
fuel_time_pd.head(1000)

,tracking_time,fuel_level,angle,altitude


In [ ]:
px.line(data_frame=fuel_time_pd,x="tracking_time",y="fuel_level",text="altitude")

#**Cleaning Data Functions**

In [18]:
def increasingtPts():
    # Initialisation d'une liste pour stocker les points d'augmentation de carburant significative
    pts = []

    for i in range(fuel_time_pd['fuel_level'].size - 1):
        # Parcourir les indices du DataFrame

        if fuel_time_pd['fuel_level'].values[i] < fuel_time_pd['fuel_level'].values[i + 1]:
            # Vérifier si le niveau de carburant à l'indice i est inférieur au niveau à l'indice suivant

            if (fuel_time_pd['fuel_level'].values[i + 1] - fuel_time_pd['fuel_level'].values[i]) >= 7:
                # Vérifier si la différence entre les niveaux de carburant est supérieure ou égale à 7

                pts.append((fuel_time_pd['fuel_level'].values[i], fuel_time_pd['fuel_level'].values[i + 1]))
                # Ajouter le tuple (niveau de carburant i, niveau de carburant i+1) à la liste 'pts'

    return pts

In [19]:
#Test
increasingtPts()

[(0.0, 13.0), (6.5, 51.0), (10.0, 48.0)]

In [20]:
def avg(pts):
    sum = 0

    for i, j in pts:
        # Parcourir les paires de points dans la liste pts
        sum += j - i
        # Ajouter la différence entre le niveau de carburant j et i à la somme

    increasingAvg = sum / len(pts)
    # Calculer la moyenne en divisant la somme par le nombre de paires de points

    return increasingAvg

In [21]:
#Test
avg(increasingtPts())

31.833333333333332

In [22]:
def isIncreasingPt(x1,x2):
  return x2>x1

In [23]:
def realIncPts(dist,avgIncr):
  return dist>=avgIncr

In [24]:
def searchCorrectVal(currentIdx,data):
  i=currentIdx-1
  while i>0:
    if data[i]>=data[currentIdx]:
      return i
    i-=1
  return -1

In [25]:
def cleaningData(preparedData):
    # Calculer la moyenne des augmentations
    avgIncreasing = avg(increasingtPts())

    # Parcourir les indices du DataFrame
    for i in range(preparedData['fuel_level'].size - 1):
        x1, x2 = preparedData['fuel_level'].values[i], preparedData['fuel_level'].values[i + 1]

        # Vérifier si c'est un point d'augmentation
        if isIncreasingPt(x1, x2):
            # Calculer la différence entre les niveaux de carburant
            dist = preparedData['fuel_level'].values[i + 1] - preparedData['fuel_level'].values[i]

            # Si l'augmentation est inférieure à la moyenne d'augmentation attendue => valeur incorrecte
            if not realIncPts(dist, avgIncreasing):
                # Trouver l'indice correct précédent
                idx = searchCorrectVal(i + 1, preparedData['fuel_level'].values)
                nbrPoints = i - idx

                # Éviter certains cas
                if nbrPoints < 1500:
                    #print(preparedData['tracking_time'].values[idx+1],preparedData['tracking_time'].values[i+1])
                    # Remplacer les valeurs
                    preparedData.loc[idx + 1:i, 'fuel_level'] = preparedData['fuel_level'].values[i + 1]

    # Retirer les lignes initiales avec des valeurs de carburant nulles
    preparedData = preparedData.iloc[len(preparedData[preparedData.fuel_level == 0]):,]

    return preparedData


In [26]:
#test
cleanData=cleaningData(preparedData=loadData(LV_data,matricule="78541.B.40 LV"))
# cleanData=cleaningData(preparedData=loadData(LV_data,matricule="10535-A-36 LV",code="89",percent=True,sizeTotale=400))

In [27]:
cleanData.head()

,tracking_time,fuel_level
0,2023-06-19 00:04:01,13.0
1,2023-06-19 00:14:02,13.0
2,2023-06-19 00:24:03,13.0
3,2023-06-19 00:34:04,13.0
4,2023-06-19 00:44:05,13.0


In [28]:
#plot cleaning data
px.line(data_frame=cleanData,x="tracking_time",y="fuel_level",title="Clening Data Plot")

#**Fuel Consumed Calculation**

\begin{equation}
  TotalFuelConsumed=\sum_{i=1}^{n}ConsumedValue_{i}
\end{equation}

\begin{equation}
  ConsumedValue_i=(refuelLevel - initialLevel)
\end{equation}

In [29]:
def calculateConsumedFuel(clean_data):
    data = clean_data
    data['fuel_diff'] = data['fuel_level'].diff()
    # Calculer la différence entre les niveaux de carburant consécutifs
    # Si la différence est positive (augmentation de carburant), nous la définissons à 0
    data.loc[data['fuel_diff'] > 0, 'fuel_diff'] = 0
    # Le carburant consommé total est la somme absolue des différences de carburant
    total_fuel_consumed = abs(data['fuel_diff'].sum())
    return total_fuel_consumed

In [30]:
#Test Algorithm
print(f'Total fuel consumed: {calculateConsumedFuel(cleanData)} L')

Total fuel consumed: 57.5 L


In [31]:
#Manual calculation
total_fuel_consumed_M = (13-6.5)+(51-10)+(50-40)
print(f'Total fuel consumed: {total_fuel_consumed_M} L')

Total fuel consumed: 57.5 L


# Web API

In [ ]:
# !pip install fastapi
# !pip install uvicorn
# !pip install requests
# !pip install pyngrok
# !pip install nest-asyncio

In [ ]:
# from fastapi import FastAPI
# import uvicorn
# from pyngrok import ngrok
# from fastapi.middleware.cors import CORSMiddleware

In [ ]:
# app = FastAPI()

In [ ]:
# origins = ["*"]

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins = origins,
#     allow_credentials = True,
#     allow_methods = ["*"],
#     allow_headers = ["*"]
# )

In [ ]:
# @app.post('/data_cleaning')
# def cleaning_service(data):
